In [ ]:
# default_exp annotation.via_adapter

In [ ]:
# hide
from nbdev.showdoc import *

In [ ]:
# export
import json
import csv
import shutil
import logging
from os.path import join, splitext, getsize, basename, isfile
from aiforce.core import assign_arg_prefix
from aiforce.annotation.core import Annotation, AnnotationAdapter, Region, RegionShape, parse_region_shape, SubsetType
from aiforce.io.core import create_folder

In [ ]:
# hide
%reload_ext autoreload
%autoreload 2

In [ ]:
# export

# The default annotation file name to use
DEFAULT_ANNOTATIONS_FILE = 'via_region_data.json'
# The default category label key to use
DEFAULT_CATEGORY_LABEL_KEY = 'category'
# The supported VIA versions
SUPPORTED_VERSIONS = [1, 2]
# The CSV headers for VIA version 1
CSV_FIELDNAMES_V1 = ['#filename', 'file_size', 'file_attributes', 'region_count', 'region_id', 'region_shape_attributes',
                     'region_attributes']
# The CSV headers for VIA version 2
CSV_FIELDNAMES_V2 = ['filename'] + CSV_FIELDNAMES_V1[1:]
# The JSON headers for VIA version 2
JSON_FIELDNAMES_V2 = {'filename': '', 'size': 0, 'regions': '[]', 'file_attributes': '{}'}
# The JSON headers for VIA version 1
JSON_FIELDNAMES_V1 = dict(JSON_FIELDNAMES_V2)
JSON_FIELDNAMES_V1.update({'regions': '{}', 'fileref': '', 'base64_img_data': ''})

In [ ]:
# export
logger = logging.getLogger(__name__)

# VIA Annotation Adapter
> VIA annotation adapter. To use the VIA annotation tool, refer to the [Homepage](http://www.robots.ox.ac.uk/~vgg/software/via/).

Current supported annotations:
- circle
- ellipse
- point
- polygon
- polyline
- rectangle

Current supported annotation tool versions:
- [VIA version 1 (v1.0.6)](https://www.robots.ox.ac.uk/~vgg/software/via/via-1.0.6.html)
- [VIA version 2 (v2.0.11)](https://www.robots.ox.ac.uk/~vgg/software/via/via.html)

The adapter has the following parameters:
- `--path`: the path to the base folder containing the annotations (e.g.: *data/image_object_detection/my_collection*)
- `--version`: The version format to read or write the anotations. If not set, parse the annotation-format on read, use version *2* on write annotation
- `--categories_file_name`: The path to the categories file if not set, default to *categories.txt*
- `--annotations_file_name`: The name of annotations file. If not set, default to *via_region_data.json*
- `--category_label_key`: The key, the category label key is stored in the annotation file, default to *category*.

In [ ]:
# export
class VIAAnnotationAdapter(AnnotationAdapter):
    """
    Adapter to read and write annotations in the VIA annotation.
    """

    def __init__(self, path, version=None, categories_file_name=None, annotations_file_name=None,
                 category_label_key=None):
        """
        VIA Adapter to read and write annotations.
        `path`: the folder containing the annotations
        `version`: the VIA version to use
        `categories_file_name`: the name of the categories file
        `annotations_file_name`: the name of annotations file
        `category_label_key`: the key of the category label
        """
        super().__init__(path, categories_file_name)
        self.version = version
        if annotations_file_name is None:
            self.annotations_file_name = DEFAULT_ANNOTATIONS_FILE
        else:
            self.annotations_file_name = annotations_file_name

        if category_label_key is None:
            self.category_label_key = DEFAULT_CATEGORY_LABEL_KEY
        else:
            self.category_label_key = category_label_key

    @classmethod
    def argparse(cls, prefix=None):
        """
        Returns the argument parser containing argument definition for command line use.
        `prefix`: a parameter prefix to set, if needed
        return: the argument parser
        """
        parser = super(VIAAnnotationAdapter, cls).argparse(prefix=prefix)
        parser.add_argument(assign_arg_prefix('--version', prefix),
                            dest="version",
                            type=int,
                            help="The version format to read or write the anotations.",
                            choices=SUPPORTED_VERSIONS,
                            default=None)
        parser.add_argument(assign_arg_prefix('--annotations_file_name', prefix),
                            dest="annotations_file_name",
                            help="The name of annotations file.",
                            default=None)
        parser.add_argument(assign_arg_prefix('--category_label_key', prefix),
                            dest="category_label_key",
                            help="The key of the category label.",
                            default=None)

        return parser

    def read_annotations(self, subset_type=SubsetType.TRAINVAL):
        """
        Reads a VIA annotations file.
        Supports JSON and CSV file format.
        `subset_type`: the subset type to read
        return: the annotations as dictionary
        """
        path = join(self.path, str(subset_type))
        annotations_file_name = self._annotation_file_name_suffix_handling(subset_type)
        annotations_file_path = join(self.path, annotations_file_name)
        logger.info('Read file sources from {}'.format(path))
        logger.info('Read annotations from {}'.format(annotations_file_path))

        file_extension = splitext(annotations_file_name)[1]

        if file_extension.lower() == '.json':
            logger.info('Read VIA annotations in JSON format')
            annotations = self._read_annotations_json(path, annotations_file_path)
        elif file_extension.lower() == '.csv':
            logger.info('Read VIA annotations in CSV format')
            annotations = self._read_annotations_csv(path, annotations_file_path)
        else:
            message = 'Unsupported annotation format at {}'.format(annotations_file_path)
            logger.error(message)
            raise ValueError(message)

        return annotations

    def _read_annotations_csv(self, path, annotations_file_path):
        """
        Reads a VIA CSV annotations file.
        `path`: the path to read file sources from
        `annotation_file_path`: the path to the annotations file
        return: the annotations as dictionary
        """
        annotations = {}
        with open(annotations_file_path, newline='') as csv_file:
            reader = csv.DictReader(csv_file)

            # try parse the version if not set explicitly
            if self.version is None:
                logger.info('VIA version not set, try to detect')
                # default is v2
                self.version = 1 if reader.fieldnames[0] == CSV_FIELDNAMES_V1[0] else 2

            logger.info('Use VIA version {}'.format(self.version))

            fieldnames = CSV_FIELDNAMES_V1 if self.version == 1 else CSV_FIELDNAMES_V2
            skipped_annotations = []
            for row in reader:
                file_path = join(path, row[fieldnames[0]])
                if not isfile(file_path):
                    logger.warning("{}: Source file not found, skip annotation.".format(file_path))
                    skipped_annotations.append(file_path)
                    continue

                annotation_id = "{}{}".format(row[fieldnames[0]], row[fieldnames[1]])

                if annotation_id not in annotations:
                    annotations[annotation_id] = Annotation(annotation_id=annotation_id, file_path=file_path)

                if int(row[fieldnames[3]]) > 0:
                    annotation = annotations[annotation_id]
                    region_shape_attributes = json.loads(row[fieldnames[5]])
                    region = self._parse_region_shape_attributes(region_shape_attributes)
                    region_attributes = json.loads(row[fieldnames[6]])
                    category = None
                    if region_attributes and self.category_label_key in region_attributes:
                        category = region_attributes[self.category_label_key]
                    region.labels = [category] if category else []
                    annotation.regions.append(region)

        logger.info('Finished read annotations')
        logger.info('Annotations read: {}'.format(len(annotations)))
        if skipped_annotations:
            logger.info('Annotations skipped: {}'.format(len(skipped_annotations)))

        return annotations

    def _read_annotations_json(self, path, annotations_file_path):
        """
        Reads a VIA JSON annotations file.
        `path`: the path to read file sources from
        `annotation_file_path`: the path to the annotations file
        return: the annotations as dictionary
        """
        annotations = {}
        with open(annotations_file_path) as json_file:
            via_annotations = json.load(json_file)

            # try parse the version if not set explicitly
            if self.version is None:
                logger.info('VIA version not set, try to detect')
                first_value = next(iter(via_annotations.values()))
                # default is v2
                self.version = 1 if first_value and isinstance(first_value['regions'], dict) else 2

            logger.info('Use VIA version {}'.format(self.version))

            skipped_annotations = []
            for data in via_annotations.values():
                file_path = join(path, data['filename'])
                if not isfile(file_path):
                    logger.warning("{}: Source file not found, skip annotation.".format(file_path))
                    skipped_annotations.append(file_path)
                    continue

                annotation_id = "{}{}".format(data['filename'], data['size'])

                if annotation_id not in annotations:
                    annotations[annotation_id] = Annotation(annotation_id=annotation_id, file_path=file_path)

                annotation = annotations[annotation_id]

                regions = data['regions'].values() if self.version == 1 else data['regions']
                for region_data in regions:
                    region_shape_attributes = region_data['shape_attributes']
                    region = self._parse_region_shape_attributes(region_shape_attributes)
                    region_attributes = region_data['region_attributes']
                    category = None
                    if region_attributes and self.category_label_key in region_attributes:
                        category = region_attributes[self.category_label_key]
                    region.labels = [category] if category else []
                    annotation.regions.append(region)

        logger.info('Finished read annotations')
        logger.info('Annotations read: {}'.format(len(annotations)))
        if skipped_annotations:
            logger.info('Annotations skipped: {}'.format(len(skipped_annotations)))

        return annotations

    def write_annotations(self, annotations, subset_type=SubsetType.TRAINVAL):
        """
        Writes a VIA annotations file and copy the corresponding source files.
        Supports JSON and CSV file format.
        Format is inferred from the annotations_file setting.
        `annotations`: the annotations as dictionary
        `subset_type`: the subset type to write
        return: a list of written target file paths
        """
        path = join(self.path, str(subset_type))
        target_folder = create_folder(path)
        annotations_file_name = self._annotation_file_name_suffix_handling(subset_type)
        annotations_file_path = join(self.path, annotations_file_name)
        logger.info('Write file sources to {}'.format(target_folder))
        logger.info('Write annotations to {}'.format(annotations_file_path))

        file_extension = splitext(annotations_file_name)[1]

        if file_extension.lower() == '.json':
            logger.info('Write VIA annotations in JSON format')
            copied_files = self._write_annotations_json(path, annotations_file_path, annotations)
        elif file_extension.lower() == '.csv':
            logger.info('Write VIA annotations in CSV format')
            copied_files = self._write_annotations_csv(path, annotations_file_path, annotations)
        else:
            message = 'Unsupported annotation format at {}'.format(annotations_file_path)
            logger.error(message)
            raise ValueError(message)
        return copied_files

    def _write_annotations_csv(self, path, annotations_file_path, annotations):
        """
        Writes a VIA CSV annotations file and copy the corresponding source files.
        `path`: the path to write file sources into
        `annotation_file_path`: the path to the annotations file
        `annotations`: the annotations to write
        return: a list of written target file paths
        """
        if self.version is None:
            logger.info('VIA version not set, set to default version')
            # default is v2
            self.version = 2

        logger.info('Use VIA version {}'.format(self.version))

        fieldnames = CSV_FIELDNAMES_V1 if self.version == 1 else CSV_FIELDNAMES_V2

        with open(annotations_file_path, 'w', newline='') as csv_file:
            writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
            writer.writeheader()

            skipped_annotations = []
            copied_files = []
            for annotation in annotations.values():
                target_file = join(path, basename(annotation.file_path))

                if not isfile(annotation.file_path):
                    logger.warning("{}: Source file not found, skip annotation.".format(annotation.file_path))
                    skipped_annotations.append(annotation.file_path)
                    continue
                if isfile(target_file):
                    logger.warning("{}: Target file already exist, skip annotation.".format(annotation.file_path))
                    skipped_annotations.append(annotation.file_path)
                    continue

                file_size = getsize(annotation.file_path)
                file_name = basename(annotation.file_path)
                for index, region in enumerate(annotation.regions):
                    region_shape_attributes = self._create_region_shape_attributes(region)
                    region_attributes = {
                        self.category_label_key: ' '.join(region.labels) if len(region.labels) else ''
                    }
                    values = [file_name, file_size, '{}', len(annotation.regions), str(index),
                              json.dumps(region_shape_attributes), json.dumps(region_attributes)]
                    writer.writerow(dict(zip(fieldnames, values)))

                # copy the file
                shutil.copy2(annotation.file_path, target_file)
                copied_files.append(target_file)

        logger.info('Finished write annotations')
        logger.info('Annotations written: {}'.format(len(annotations) - len(skipped_annotations)))
        if skipped_annotations:
            logger.info('Annotations skipped: {}'.format(len(skipped_annotations)))
        return copied_files

    def _write_annotations_json(self, path, annotations_file_path, annotations):
        """
        Writes a VIA JSON annotations file and copy the corresponding source files.
        `path`: the path to write file sources into
        `annotation_file_path`: the path to the annotations file
        `annotations`: the annotations to write
        return: a list of written target file paths
        """
        json_annotations = {}
        skipped_annotations = []
        copied_files = []

        if self.version is None:
            logger.info('VIA version not set, set to default version')
            # default is v2
            self.version = 2

        logger.info('Use VIA version {}'.format(self.version))

        fieldnames = JSON_FIELDNAMES_V1 if self.version == 1 else JSON_FIELDNAMES_V2

        for annotation in annotations.values():
            target_file = join(path, basename(annotation.file_path))

            if not isfile(annotation.file_path):
                logger.warning("{}: Source file not found, skip annotation.".format(annotation.file_path))
                skipped_annotations.append(annotation.file_path)
                continue
            if isfile(target_file):
                logger.warning("{}: Target file already exist, skip annotation.".format(annotation.file_path))
                skipped_annotations.append(annotation.file_path)
                continue

            file_size = getsize(annotation.file_path)
            file_name = basename(annotation.file_path)
            file_id = '{:s}{:d}'.format(file_name, file_size)
            regions = [{
                'shape_attributes': self._create_region_shape_attributes(region),
                'region_attributes': {
                    self.category_label_key: ' '.join(region.labels) if len(region.labels) else ''
                }
            } for region in annotation.regions]

            json_annotation = dict(fieldnames)
            json_annotation.update({
                'size': file_size,
                'filename': file_name,
                "regions": {str(i): v for i, v in enumerate(regions)} if self.version == 1 else regions
            })
            json_annotations[file_id] = json_annotation

            # copy the file
            shutil.copy2(annotation.file_path, target_file)
            copied_files.append(target_file)

        with open(annotations_file_path, 'w') as json_file:
            json.dump(json_annotations, json_file)

        logger.info('Finished write annotations')
        logger.info('Annotations written: {}'.format(len(annotations) - len(skipped_annotations)))
        if skipped_annotations:
            logger.info('Annotations skipped: {}'.format(len(skipped_annotations)))
        return copied_files

    def _annotation_file_name_suffix_handling(self, subset_type):
        """
        Handle annotations file name based on the subset type.
        `subset_type`: the subset type to handle
        return: the annotations file name
        """
        file_name, ext = splitext(self.annotations_file_name)
        if subset_type in [SubsetType.TRAIN, SubsetType.VAL] and not file_name.endswith(str(subset_type)):
            suffix = "_{}".format(str(subset_type))
            return "{}{}{}".format(file_name, suffix, ext)
        return self.annotations_file_name

    def _parse_region_shape_attributes(self, region_shape_attributes):
        """
        Parse region shape attributes.
        `region_shape_attributes`: the region shape attributes as dictionary
        return: the corresponding annotation
        """
        if not region_shape_attributes:
            return Annotation()

        region_shape = parse_region_shape(region_shape_attributes['name'])
        points_x = None
        points_y = None
        radius_x = 0
        radius_y = 0
        rotation = 0
        if region_shape == RegionShape.CIRCLE:
            points_x = [region_shape_attributes['cx']]
            points_y = [region_shape_attributes['cy']]
            radius_x = region_shape_attributes['r']
            radius_y = region_shape_attributes['r']
        elif region_shape == RegionShape.ELLIPSE:
            points_x = [region_shape_attributes['cx']]
            points_y = [region_shape_attributes['cy']]
            radius_x = region_shape_attributes['rx']
            radius_y = region_shape_attributes['ry']
            rotation = 0 if self.version == 1 else region_shape_attributes['theta']
        elif region_shape == RegionShape.POINT:
            points_x = [region_shape_attributes['cx']]
            points_y = [region_shape_attributes['cy']]
        elif region_shape == RegionShape.POLYGON or region_shape == RegionShape.POLYLINE:
            points_x = region_shape_attributes['all_points_x']
            points_y = region_shape_attributes['all_points_y']
        elif region_shape == RegionShape.RECTANGLE:
            x = region_shape_attributes['x']
            y = region_shape_attributes['y']
            width = region_shape_attributes['width']
            height = region_shape_attributes['height']
            points_x = [x, x + width]
            points_y = [y, y + height]
        return Region(shape=region_shape, points_x=points_x, points_y=points_y, radius_x=radius_x, radius_y=radius_y,
                      rotation=rotation)

    def _create_region_shape_attributes(self, region: Region):
        """
        Create region shape attributes.
        `region`: the region to create region shape attributes from
        return: the corresponding region shape attributes as dictionary
        """
        region_shape_attributes = {
            "name": str(region.shape),

        }
        c_x = region.points_x[0] if len(region.points_x) else 0
        c_y = region.points_y[0] if len(region.points_y) else 0

        if region.shape == RegionShape.CIRCLE:
            region_shape_attributes['cx'] = c_x
            region_shape_attributes['cy'] = c_y
            region_shape_attributes['r'] = max(region.radius_x, region.radius_y)
        elif region.shape == RegionShape.ELLIPSE:
            region_shape_attributes['cx'] = c_x
            region_shape_attributes['cy'] = c_y
            region_shape_attributes['rx'] = region.radius_x
            region_shape_attributes['ry'] = region.radius_y
            # version 1 do not support ellipse rotation
            if self.version == 1 and region.rotation:
                logger.warning("VIA version {} do not support rotation of shape {}. Rotation is set to 0.".format(
                    self.version,
                    region.shape,
                ))
            else:
                region_shape_attributes['theta'] = region.rotation
        elif region.shape == RegionShape.POINT:
            region_shape_attributes['cx'] = c_x
            region_shape_attributes['cy'] = c_y
        elif region.shape == RegionShape.POLYGON or region.shape == RegionShape.POLYLINE:
            region_shape_attributes['all_points_x'] = region.points_x
            region_shape_attributes['all_points_y'] = region.points_y
        elif region.shape == RegionShape.RECTANGLE:
            region_shape_attributes['x'] = region.points_x[0]
            region_shape_attributes['y'] = region.points_y[0]
            region_shape_attributes['width'] = region.points_x[1] - region.points_x[0]
            region_shape_attributes['height'] = region.points_y[1] - region.points_y[0]
        return region_shape_attributes


In [ ]:
show_doc(VIAAnnotationAdapter.list_files)
show_doc(VIAAnnotationAdapter.read_annotations)
show_doc(VIAAnnotationAdapter.read_categories)
show_doc(VIAAnnotationAdapter.write_files)
show_doc(VIAAnnotationAdapter.write_annotations)
show_doc(VIAAnnotationAdapter.write_categories)
show_doc(VIAAnnotationAdapter.argparse)

<h4 id="AnnotationAdapter.list_files" class="doc_header"><code>AnnotationAdapter.list_files</code><a href="https://github.com/AI-Force/aiforce/tree/master/aiforce/annotation/core.py#L200" class="source_link" style="float:right">[source]</a></h4>

> <code>AnnotationAdapter.list_files</code>(**`subset_type`**=*`<SubsetType.TRAINVAL: 'trainval'>`*)

List all physical files in a sub-set.
`subset_type`: the subset type to list
return: a list of file paths if subset type exist, else an empty list

<h4 id="VIAAnnotationAdapter.read_annotations" class="doc_header"><code>VIAAnnotationAdapter.read_annotations</code><a href="__main__.py#L54" class="source_link" style="float:right">[source]</a></h4>

> <code>VIAAnnotationAdapter.read_annotations</code>(**`subset_type`**=*`<SubsetType.TRAINVAL: 'trainval'>`*)

Reads a VIA annotations file.
Supports JSON and CSV file format.
`subset_type`: the subset type to read
return: the annotations as dictionary

<h4 id="AnnotationAdapter.read_categories" class="doc_header"><code>AnnotationAdapter.read_categories</code><a href="https://github.com/AI-Force/aiforce/tree/master/aiforce/annotation/core.py#L235" class="source_link" style="float:right">[source]</a></h4>

> <code>AnnotationAdapter.read_categories</code>()

Read categories.
return: a list of category names

<h4 id="AnnotationAdapter.write_files" class="doc_header"><code>AnnotationAdapter.write_files</code><a href="https://github.com/AI-Force/aiforce/tree/master/aiforce/annotation/core.py#L209" class="source_link" style="float:right">[source]</a></h4>

> <code>AnnotationAdapter.write_files</code>(**`file_paths`**, **`subset_type`**=*`<SubsetType.TRAINVAL: 'trainval'>`*)

Write physical files in a sub-set.
`file_paths`: a list of file paths to write
`subset_type`: the subset type to write into
return: a list of written target file paths

<h4 id="VIAAnnotationAdapter.write_annotations" class="doc_header"><code>VIAAnnotationAdapter.write_annotations</code><a href="__main__.py#L187" class="source_link" style="float:right">[source]</a></h4>

> <code>VIAAnnotationAdapter.write_annotations</code>(**`annotations`**, **`subset_type`**=*`<SubsetType.TRAINVAL: 'trainval'>`*)

Writes a VIA annotations file and copy the corresponding source files.
Supports JSON and CSV file format.
Format is inferred from the annotations_file setting.
`annotations`: the annotations as dictionary
`subset_type`: the subset type to write
return: a list of written target file paths

<h4 id="AnnotationAdapter.write_categories" class="doc_header"><code>AnnotationAdapter.write_categories</code><a href="https://github.com/AI-Force/aiforce/tree/master/aiforce/annotation/core.py#L254" class="source_link" style="float:right">[source]</a></h4>

> <code>AnnotationAdapter.write_categories</code>(**`categories`**)

Write categories.
`categories`: a list of category names

<h4 id="VIAAnnotationAdapter.argparse" class="doc_header"><code>VIAAnnotationAdapter.argparse</code><a href="__main__.py#L29" class="source_link" style="float:right">[source]</a></h4>

> <code>VIAAnnotationAdapter.argparse</code>(**`prefix`**=*`None`*)

Returns the argument parser containing argument definition for command line use.
`prefix`: a parameter prefix to set, if needed
return: the argument parser

## Test the Adapter

Define the test data to use in tests. 
We declare 2 annotations, one with all supported region shapes set and one with no region set at all.
The we define the result formats in CSV and JSON for VIA version 1 and 2.

In [ ]:
import unittest
from unittest.mock import patch, mock_open
from fastcore.test import *

In [ ]:
CATEGORIES_TEST_DATA = "foo\nbar"
TEST_FILEPATH = "testpath"
ANNOTATION_TEST_DATA = {
    "foo.jpg12": Annotation(annotation_id="foo.jpg12", file_path=join(TEST_FILEPATH, "foo.jpg"), regions=[
        Region(shape=RegionShape.CIRCLE, points_x=[20], points_y=[21], radius_x=5, radius_y=5, labels=['foo']),
        Region(shape=RegionShape.ELLIPSE, points_x=[30], points_y=[31], radius_x=10, radius_y=15, rotation=0.6, labels=['bar']),
        Region(shape=RegionShape.POINT, points_x=[40], points_y=[41], labels=['foo']),
        Region(shape=RegionShape.POLYGON, points_x=[50, 60, 70, 80], points_y=[51, 61, 71, 81], labels=['bar']),
        Region(shape=RegionShape.POLYLINE, points_x=[52, 62, 72, 82], points_y=[53, 63, 73, 83], labels=['foo']),
        Region(shape=RegionShape.RECTANGLE, points_x=[90, 100], points_y=[91, 111], labels=['bar']),
    ]),
    "bar.jpg34": Annotation(annotation_id="bar.jpg34", file_path=join(TEST_FILEPATH, "bar.jpg"))
}
# The CSV test data for VIA version 1
DATA_CSV_V1 = """#filename,file_size,file_attributes,region_count,region_id,region_shape_attributes,region_attributes
foo.jpg,12,"{}",6,0,"{""name"":""circle"",""cx"":20,""cy"":21,""r"":5}","{""category"":""foo""}"
foo.jpg,12,"{}",6,1,"{""name"":""ellipse"",""cx"":30,""cy"":31,""rx"":10,""ry"":15}","{""category"":""bar""}"
foo.jpg,12,"{}",6,2,"{""name"":""point"",""cx"":40,""cy"":41}","{""category"":""foo""}"
foo.jpg,12,"{}",6,3,"{""name"":""polygon"",""all_points_x"":[50, 60, 70, 80],""all_points_y"":[51, 61, 71, 81]}","{""category"":""bar""}"
foo.jpg,12,"{}",6,4,"{""name"":""polyline"",""all_points_x"":[52, 62, 72, 82],""all_points_y"":[53, 63, 73, 83]}","{""category"":""foo""}"
foo.jpg,12,"{}",6,5,"{""name"":""rect"",""x"":90,""y"":91,""width"":10,""height"":20}","{""category"":""bar""}"
bar.jpg,34,"{}",0,0,"{}","{}"
"""
# The CSV test data for VIA version 2
DATA_CSV_V2 = """filename,file_size,file_attributes,region_count,region_id,region_shape_attributes,region_attributes
foo.jpg,12,"{}",6,0,"{""name"":""circle"",""cx"":20,""cy"":21,""r"":5}","{""category"":""foo""}"
foo.jpg,12,"{}",6,1,"{""name"":""ellipse"",""cx"":30,""cy"":31,""rx"":10,""ry"":15,""theta"":0.6}","{""category"":""bar""}"
foo.jpg,12,"{}",6,2,"{""name"":""point"",""cx"":40,""cy"":41}","{""category"":""foo""}"
foo.jpg,12,"{}",6,3,"{""name"":""polygon"",""all_points_x"":[50, 60, 70, 80],""all_points_y"":[51, 61, 71, 81]}","{""category"":""bar""}"
foo.jpg,12,"{}",6,4,"{""name"":""polyline"",""all_points_x"":[52, 62, 72, 82],""all_points_y"":[53, 63, 73, 83]}","{""category"":""foo""}"
foo.jpg,12,"{}",6,5,"{""name"":""rect"",""x"":90,""y"":91,""width"":10,""height"":20}","{""category"":""bar""}"
bar.jpg,34,"{}",0,0,"{}","{}"
"""
# The JSON test data for VIA version 1
DATA_JSON_V1 = """{
"foo.jpg12":{"fileref":"","size":12,"filename":"foo.jpg","base64_img_data":"","file_attributes":{},"regions":{
"0":{"shape_attributes":{"name":"circle","cx":20,"cy":21,"r":5},"region_attributes":{"category":"foo"}},
"1":{"shape_attributes":{"name":"ellipse","cx":30,"cy":31,"rx":10,"ry":15},"region_attributes":{"category":"bar"}},
"2":{"shape_attributes":{"name":"point","cx":40,"cy":41},"region_attributes":{"category":"foo"}},
"3":{"shape_attributes":{"name":"polygon","all_points_x":[50, 60, 70, 80],"all_points_y":[51, 61, 71, 81]},"region_attributes":{"category":"bar"}},
"4":{"shape_attributes":{"name":"polyline","all_points_x":[52, 62, 72, 82],"all_points_y":[53, 63, 73, 83]},"region_attributes":{"category":"foo"}},
"5":{"shape_attributes":{"name":"rect","x":90,"y":91,"width":10,"height":20},"region_attributes":{"category":"bar"}}}},
"bar.jpg34":{"fileref":"","size":34,"filename":"bar.jpg","base64_img_data":"","file_attributes":{},"regions":{}}}"""
# The JSON test data for VIA version 2
DATA_JSON_V2 = """{
"foo.jpg12":{"fileref":"","size":12,"filename":"foo.jpg","regions":[
{"shape_attributes":{"name":"circle","cx":20,"cy":21,"r":5},"region_attributes":{"category":"foo"}},
{"shape_attributes":{"name":"ellipse","cx":30,"cy":31,"rx":10,"ry":15,"theta":0.6},"region_attributes":{"category":"bar"}},
{"shape_attributes":{"name":"point","cx":40,"cy":41},"region_attributes":{"category":"foo"}},
{"shape_attributes":{"name":"polygon","all_points_x":[50, 60, 70, 80],"all_points_y":[51, 61, 71, 81]},"region_attributes":{"category":"bar"}},
{"shape_attributes":{"name":"polyline","all_points_x":[52, 62, 72, 82],"all_points_y":[53, 63, 73, 83]},"region_attributes":{"category":"foo"}},
{"shape_attributes":{"name":"rect","x":90,"y":91,"width":10,"height":20},"region_attributes":{"category":"bar"}}],"file_attributes":{}},
"bar.jpg34":{"fileref":"","size":34,"filename":"bar.jpg","regions":{},"file_attributes":{}}}"""

In [ ]:
adapter = VIAAnnotationAdapter(TEST_FILEPATH)

with patch('__main__.isfile', return_value=True):
    with patch('__main__.open'.format(__name__), new=mock_open(read_data=DATA_JSON_V2)) as _file:
        actual = adapter.read_annotations()
        _file.assert_called_once_with(join(TEST_FILEPATH, 'via_region_data.json'))
test_eq(actual, ANNOTATION_TEST_DATA)
actual

AssertionError: ==:
{'foo.jpg12': <aiforce.annotation.core.Annotation object at 0x7fc190a9acd0>, 'bar.jpg34': <aiforce.annotation.core.Annotation object at 0x7fc190a9a790>}
{'foo.jpg12': <aiforce.annotation.core.Annotation object at 0x7fc18059b850>, 'bar.jpg34': <aiforce.annotation.core.Annotation object at 0x7fc18059b8d0>}

In [ ]:
# hide

# for generating scripts from notebook directly
from nbdev.export import notebook2script
notebook2script()